# Used Bike Prices — Feature Engineering & EDA

**Project:** Used Bike Prices - Feature Engineering and EDA  
**Author:** _Your Name_  
**Tools:** VS Code, Jupyter, Python (pandas, numpy, matplotlib, scikit-learn)  

This notebook performs a comprehensive cleaning + EDA + baseline modeling pipeline on the provided dataset.
It follows the structure in the 69-page guide you shared and mirrors key transformations (parsing text fields,
deriving `cc` and `brand`, fixing `mileage`, `power`, `kms_driven`, etc.), with clear cells you can run end‑to‑end.


## Table of Contents
1. [Setup](#setup)
2. [Load Data](#load)
3. [Quick Audit](#audit)
4. [Cleaning & Standardization](#clean)
   - model_name → `brand`, `cc`, `model_year` sanity
   - `mileage` numeric
   - `power` to bhp
   - `kms_driven` numeric
   - `owner`, `location` tidy
5. [Feature Engineering](#fe)
   - Age of bike
   - Ownership dummies, Brand dummies
6. [Exploratory Data Analysis](#eda)
7. [Train/Test Split & Baseline Models](#model)
8. [Save Artifacts](#save)


## 1) Setup <a id='setup'></a>

In [ ]:
# Core
import pandas as pd
import numpy as np
import re
from pathlib import Path
from datetime import datetime

# Viz (stick to matplotlib per guidance)
import matplotlib.pyplot as plt

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Display
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 120)
print('Libraries imported.')

## 2) Load Data <a id='load'></a>

In [ ]:
DATA_PATH = Path('/mnt/data/bikes.csv')  # update if needed
assert DATA_PATH.exists(), f'Missing file: {DATA_PATH}'

raw = pd.read_csv(DATA_PATH)
print(raw.shape)
raw.head(10)

## 3) Quick Audit <a id='audit'></a>

In [ ]:
raw.info()
raw.describe(include='all').T

## 4) Cleaning & Standardization <a id='clean'></a>

### 4.1 Make a working copy

In [ ]:
df = raw.copy()

# Normalize column names
df.columns = [c.strip().lower() for c in df.columns]
df.head(2)

### 4.2 Extract `brand` and `cc` from `model_name`

In [ ]:
def extract_brand(name: str) -> str:
    try:
        return str(name).strip().split()[0]
    except Exception:
        return np.nan

def extract_cc(name: str):
    # Parse things like 'Royal Enfield Classic 350cc 2016' or 'Yamaha R15 150 cc'
    # Returns numeric cc if found, else NaN.
    s = str(name).lower()
    m = re.search(r'(\d{2,4})\s*cc', s)
    if m:
        try:
            return float(m.group(1))
        except:
            return np.nan
    return np.nan

df['brand'] = df['model_name'].apply(extract_brand)
df['cc'] = df['model_name'].apply(extract_cc)

# Standardize brand capitalization
df['brand'] = df['brand'].str.title()

df[['model_name','brand','cc']].head(10)

### 4.3 Clean `mileage` → numeric (kmpl)

In [ ]:
def parse_mileage(mil):
    # Accepts variants: '\n\n 35 kmpl', '80 Kmpl', '28 Kms' (bad), '', 'nan', 'liquid cooled'
    if pd.isna(mil):
        return np.nan
    s = str(mil).strip().lower()
    s = s.replace('kmpl','').replace('km/l','').replace('km / l','')
    s = s.replace('kms','')  # bad unit occasionally
    s = s.replace('\n',' ').replace('\r',' ')
    s = s.replace('liquid cooled','')  # treat as missing
    m = re.search(r'(-?\d+(\.\d+)?)', s)  # first numeric token
    return float(m.group(1)) if m else np.nan

df['mileage_kmpl'] = df['mileage'].apply(parse_mileage)
df['mileage_kmpl'].describe()

### 4.4 Clean `power` → numeric bhp

In [ ]:
def parse_power_bhp(p):
    # Accepts: '19 bhp', '43 bhp @ 9000 rpm', '15.2 Bhp @ 8500 rpm', '50 kW', '60 PS', '20 hp'
    if pd.isna(p):
        return np.nan
    s = str(p).strip().lower()
    s = s.split('@')[0].strip()  # drop rpm part
    # kw → bhp
    if 'kw' in s:
        m = re.search(r'(\d+(\.\d+)?)', s)
        return float(m.group(1)) * 1.341 if m else np.nan
    # ps → bhp
    if 'ps' in s:
        m = re.search(r'(\d+(\.\d+)?)', s)
        return float(m.group(1)) * 0.99 if m else np.nan
    # hp → bhp (approx)
    if 'hp' in s and 'bhp' not in s:
        m = re.search(r'(\d+(\.\d+)?)', s)
        return float(m.group(1)) * 0.986 if m else np.nan
    # default: bhp present or unitless number
    m = re.search(r'(\d+(\.\d+)?)', s)
    return float(m.group(1)) if m else np.nan

df['power_bhp'] = df['power'].apply(parse_power_bhp)
df['power_bhp'].describe()

### 4.5 Clean `kms_driven` → numeric km

In [ ]:
def parse_kms(x):
    # Accepts '17000 Km', 'Mileage 28 Kms', '30000 Km', 'yes' (bad)
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in {'yes','mileage'}:
        return np.nan
    m = re.search(r'(\d+(\.\d+)?)', s)
    return float(m.group(1)) if m else np.nan

df['kms'] = df['kms_driven'].apply(parse_kms)
df['kms'] = df['kms'].fillna(df['kms'].median())
df['kms'].describe()

### 4.6 Owner & Location tidy

In [ ]:
def normalize_owner(o):
    if pd.isna(o): return np.nan
    s = str(o).strip().lower()
    if 'first' in s: return 'first'
    if 'second' in s: return 'second'
    if 'third' in s: return 'third'
    return 'fourth_or_more'

df['owner_cat'] = df['owner'].apply(normalize_owner)
df['location'] = df['location'].astype(str).str.strip().str.replace('\s+', ' ', regex=True).str.title()

df[['owner', 'owner_cat', 'location']].head(10)

### 4.7 Final type fixes & target

In [ ]:
for col in ['model_year', 'mileage_kmpl', 'power_bhp', 'kms', 'cc']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df['price'] = pd.to_numeric(df['price'], errors='coerce')

print(df[['model_year','brand','cc','mileage_kmpl','power_bhp','kms','price']].describe())

## 5) Feature Engineering <a id='fe'></a>

In [ ]:
from datetime import datetime
CURRENT_YEAR = datetime.now().year
df['bike_age'] = CURRENT_YEAR - df['model_year']

# Clip extreme outliers
df['kms_clipped'] = df['kms'].clip(lower=0, upper=df['kms'].quantile(0.99))
df['power_bhp_clipped'] = df['power_bhp'].clip(lower=0, upper=df['power_bhp'].quantile(0.99))
df['mileage_kmpl_clipped'] = df['mileage_kmpl'].clip(lower=0, upper=df['mileage_kmpl'].quantile(0.99))

# Categorical encodings
owner_dummies = pd.get_dummies(df['owner_cat'], prefix='owner', drop_first=False)

brand_top = df['brand'].value_counts().index[:15]
df['brand_top'] = np.where(df['brand'].isin(brand_top), df['brand'], 'Other')
brand_dummies = pd.get_dummies(df['brand_top'], prefix='brand', drop_first=False)

feats = pd.concat([
    df[['bike_age','kms_clipped','power_bhp_clipped','mileage_kmpl_clipped','cc']],
    owner_dummies, brand_dummies
], axis=1)

target = df['price']
feats.shape, target.shape

## 6) Exploratory Data Analysis <a id='eda'></a>

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(target.dropna(), bins=50)
plt.title('Price Distribution')
plt.xlabel('Price (INR)')
plt.ylabel('Count')
plt.show()

for col in ['bike_age','kms_clipped','power_bhp_clipped','mileage_kmpl_clipped','cc']:
    plt.figure(figsize=(6,4))
    plt.scatter(df[col], target, s=8, alpha=0.5)
    plt.title(f'Price vs {col}')
    plt.xlabel(col)
    plt.ylabel('Price')
    plt.show()

owner_mean = df.groupby('owner_cat')['price'].mean().sort_values(ascending=False)
plt.figure(figsize=(6,4))
owner_mean.plot(kind='bar')
plt.title('Mean Price by Owner Category')
plt.xlabel('Owner Category')
plt.ylabel('Mean Price (INR)')
plt.show()

top_counts = df['brand'].value_counts().head(15)
plt.figure(figsize=(8,4))
top_counts.plot(kind='bar')
plt.title('Top 15 Brands by Listing Count')
plt.xlabel('Brand')
plt.ylabel('Count')
plt.show()

## 7) Train/Test Split & Baseline Models <a id='model'></a>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import pandas as pd

X = feats.fillna(feats.median(numeric_only=True))
y = target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate(name, model):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    rmse = mean_squared_error(y_test, pred, squared=False)
    r2 = r2_score(y_test, pred)
    return {'model': name, 'MAE': mae, 'RMSE': rmse, 'R2': r2}

results = []
results.append(evaluate('LinearRegression', LinearRegression()))
results.append(evaluate('Ridge', Ridge(alpha=1.0)))
results.append(evaluate('Lasso', Lasso(alpha=0.001, max_iter=10000)))
results.append(evaluate('RandomForest', RandomForestRegressor(n_estimators=200, random_state=42)))
results.append(evaluate('GradientBoosting', GradientBoostingRegressor(random_state=42)))

pd.DataFrame(results).sort_values('RMSE')

## 8) Save Artifacts <a id='save'></a>

In [ ]:
from pathlib import Path
OUT_DIR = Path('/mnt/data/artifacts')
OUT_DIR.mkdir(parents=True, exist_ok=True)

df.to_csv(OUT_DIR / 'used_bikes_cleaned.csv', index=False)
feats.join(y.rename('price')).to_csv(OUT_DIR / 'used_bikes_model_table.csv', index=False)

print('Saved:')
print(OUT_DIR / 'used_bikes_cleaned.csv')
print(OUT_DIR / 'used_bikes_model_table.csv')

---
## VS Code + Git Quickstart

**Repo name suggestion:** `Used-Bike-Prices-EDA`

1. Open the folder in VS Code (File → Open Folder...).  
2. Create a new Git repository:
   ```bash
   git init
   git add .
   git commit -m "Initial: EDA + Feature Engineering notebook"
   ```
3. Create a GitHub repo with the same name (on github.com).  
4. Link and push:
   ```bash
   git branch -M main
   git remote add origin https://github.com/<your-username>/Used-Bike-Prices-EDA.git
   git push -u origin main
   ```
5. Add a README with project overview, data dictionary, and results.
